<a href="https://colab.research.google.com/github/purvasingh96/Deep-learning-with-neural-networks/blob/master/Deep-learning-with-pytorch/3.%20Recurrent%20Neural%20Networks/Sentiment_analysis_via_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with an RNN
In this notebook I have implemented a RNN that performs sentiment analysis. <br>
Reason for using RNN instead of a strictly feedforward network is that we can also include information about *sequence* of words.

### Network Architecture
Below would be the architecture diagram for my sentiment analysis model - <br>
<img src="https://github.com/purvasingh96/Deep-learning-with-neural-networks/blob/master/Deep-learning-with-pytorch/3.%20Recurrent%20Neural%20Networks/images/network_diagram.png" width=40%></img>

In [0]:
import numpy as np

In [0]:
with open('sample_data/reviews.txt', 'r') as f:
  reviews = f.read()
with open('sample_data/labels.txt', 'r') as f:
  labels = f.read()

In [6]:
print(reviews[:100])
print(labels[:100])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
n


In [0]:
from string import punctuation

# get rid of punctuation
